In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import NetModules
import utils
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
def fun(x, t):
    x_bar = torch.mean(x)
    return torch.exp(x_bar * (t + 2*t**2) + 3 * t * x_bar**3)

In [3]:
class DeepSet(nn.Module):

    def __init__(self, N):
        super().__init__() # Runs initialisation of nn.Module
        self.fc1 = nn.Linear(1, 64).float()
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 128)
        self.fc5 = nn.Linear(128, 1024)
        
        self.psi1 = nn.Linear(1025, 512)
        self.psi2 = nn.Linear(512, 256)
        self.psi3 = nn.Linear(256, 1)
        
        self.N = N

    def phi(self, x):
        x = x.float()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x

    def psi(self, x):
        x = x.float()
        x = F.relu(self.psi1(x))
        x = F.relu(self.psi2(x))
        x = self.psi3(x)
        return x
        
    def forward(self, x, t):
        x = x.float()
        N = self.N
        y = torch.zeros(N, 1024)
        for i in range(N):
            y[i] = self.phi(x[i].view(-1).float())
        y = torch.sum(y, 0) / N
        y = torch.cat([y, t])
        return self.psi(y)

In [4]:
N = 10
n_data = 1000
x = torch.normal(torch.zeros(n_data, N), torch.ones(n_data, N)).float()
t = torch.tensor(np.linspace(0,1, n_data)).view(-1, 1)
synth = torch.zeros(n_data)
for i in range(0,n_data):
    synth[i] = fun(x[i, :], t[i])
synth = synth.float()

In [6]:
batches = 10
net = DeepSet(N).float()
optimizer = optim.Adam(net.parameters(), lr=0.01) # Corresponds to evything that is adjustable
criterion = nn.MSELoss()
EPOCHS = 100
outputs_epoch = []
for epoch in range(EPOCHS):
    outputs = []
    net.zero_grad()
    for i in range(batches):
        k = int(n_data /batches)
        dataX = x[i:i+k,:]
        dataY = synth[i:i+k]
        tot_loss = 0
        net.zero_grad()
        l = 0
        for j , (X, Y) in enumerate(zip(dataX, dataY)):
            output = net(X, t[l])
            loss = criterion(output, Y)
            l += 1
            outputs.append(output)
        tot_loss += loss
        tot_loss.backward() # Backpropagate the loss
        optimizer.step() # Adjusts the steps
    outputs_epoch.append(outputs)
    print('LOSS = ', tot_loss)

LOSS =  tensor(0.1014, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0136, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0032, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0055, grad_fn=<AddBackward0>)
LOSS =  tensor(3.2362e-05, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0002, grad_fn=<AddBackward0>)
LOSS =  tensor(3.4088e-05, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0002, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0005, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0003, grad_fn=<AddBackward0>)
LOSS =  tensor(4.0990e-05, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0001, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0001, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0003, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0018, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0052, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0085, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0006, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0046, grad_fn=<AddBackward0>)
LOSS =  tensor(0.0125, grad_fn=<AddBackward0>)
LOSS =  tensor(7.1043e-05, grad_fn=<AddBackward0

In [9]:
def plot_results(outputs_epoch, synth, title, k):
    plt.close()
    for i, outputs in enumerate(outputs_epoch):
        if i%k==1:
            outputs = np.array([output.detach().numpy()[0] for output in outputs])
            plt.plot(outputs, alpha=((i+1)/len(outputs_epoch))**2, label=str(i))
    outputs = np.array([output.detach().numpy()[0] for output in outputs_epoch[-1]])
    plt.plot(outputs, alpha=1, label="last")
    plt.plot(synth.detach().numpy(), label="true")
    plt.legend()
    plt.title(title)

In [11]:
plot_results(outputs_epoch, synth, "DeepSet 10 batches", 20)

In [ ]:
print(len(outputs_epoch))
print(len(outputs_epoch[0]))

In [310]:
net = DeepSet(N).float()
optimizer = optim.Adam(net.parameters(), lr=0.01) # Corresponds to evything that is adjustable
criterion = nn.MSELoss()
EPOCHS = 10
outputs_epoch = []
for epoch in range(EPOCHS):
    outputs = []
    for dataX, dataY in zip(trainloaderX, trainloaderY):
        #for j, d in enumerate(data):
        tot_loss = 0
        for j, (X, Y) in enumerate(zip(dataX, dataY)):
            net.zero_grad()
            #print(synth)
            #print(t[j].shape)
            #print(X.shape)
            output = net(X, t[j])
            outputs.append(output)
            #print(output)
            loss = criterion(output, Y)
            #print(synth, output)
            tot_loss += loss
        tot_loss.backward() # Backpropagate the loss
        optimizer.step() # Adjusts the steps
    outputs_epoch.append(outputs)
    print('LOSS = ', tot_loss)

LOSS =  tensor(401.8682, grad_fn=<AddBackward0>)


KeyboardInterrupt: 

In [286]:
print(len(trainloaderX))
print(len(dataX))


2
50


In [281]:
print(len(outputs_epoch[0]))
len(synth)

100


1000

In [311]:
for i, outputs in enumerate(outputs_epoch):
    if i%5==1:
        outputs = np.array([output.detach().numpy()[0] for output in outputs])
        plt.plot(outputs, label=str(i))
plt.plot(synth.detach().numpy(), label="true")
plt.legend()